In [1]:
import numpy as np

In [2]:
# generate random patters of ltl

# 1. unary operators G, F, R, W, X, ~
# 2. binary operators  &, |, →, ↔     (include true and false?)
# 3. variables a, b, c, d....

# generate formulas as random nodes in a tree
# for each node certain prob of of node being populated

## Generating random LTL formulas as binary trees

In [203]:
binary_operators = ['&', '|', '→', '↔']
binary_dist = np.array([0.35, 0.35, 0.2, 0.10])

unary_operators = ['G', 'F', 'R', '~']
unary_dist = np.array([0.2, 0.2, 0.2, 0.4])

variables = ['a', 'b', 'c', 'd']
variable_dist = np.array([0.4, 0.35, 0.15, 0.10]) # logarithmic probabilities

symbols = binary_operators + unary_operators + variables
symbol_dist = np.concatenate([binary_dist * len(binary_dist)/len(symbols), #P(symbol) = P(class) * P(symbol | class)
                              unary_dist * len(unary_dist)/len(symbols),
                              variable_dist * len(variable_dist)/len(symbols)]) # weight for None symbol

symbol_dist /= np.sum(symbol_dist) # making the dist sum up to 1

class LTLfomula:
    def __init__(self, symbol, depth, max_depth):
        self.symbol = symbol
        self.depth = depth
        self.max_depth = max_depth

        self.left = None
        self.right = None

        # left child
        if self.symbol not in unary_operators: # unary symbols can have only one child
            self.left = self.sample_child()
        # right child
        self.right = self.sample_child()


    def sample_child(self):
        if self.depth == self.max_depth or self.symbol in variables:
            return None
        elif self.depth == self.max_depth - 1: # the next layer is the last one -> generate variable
            return LTLfomula(sample_variable(), self.depth + 1, self.max_depth)
        else:
            return LTLfomula(sample_symbol(), self.depth + 1, self.max_depth)

def sample_variable():
    return np.random.choice(variables, size=1, p=variable_dist)[0]

def sample_symbol():
    return np.random.choice(symbols, size=1, p=symbol_dist)[0]

def generate_random_LTL(max_depth):
    while True:
        root_symbol = sample_symbol()
        if root_symbol in variables: # filter out only propositions
            continue
        tree = LTLfomula(root_symbol, depth=1, max_depth=max_depth)
        break

    return tree

## Parsing tree representation into formula

In [204]:
def insert_at(s, insert, cursor):
    return s[:cursor] + insert + s[cursor:]

def print_formula_tree(tree):
    def recursive_print_node(node, output="", cursor=0): # modified in order traversal
        if node is not None and node.symbol is not None:
            output, cursor = recursive_print_node(node.left, output, cursor)

            output = insert_at(output, node.symbol, cursor)
            cursor += 1

            if node.symbol in unary_operators:
                output = insert_at(output, "()", cursor)
                cursor += 1

            output, cursor = recursive_print_node(node.right, output, cursor)

            #if node.symbol in binary_operators:
            #    output = insert_at(output, "(", cursor)
             #   output = insert_at(output, ")", cursor)
            #    cursor += 1

        return output, cursor

    output, _ = recursive_print_node(tree)
    return output

In [205]:
for i in range(50):
    tree = generate_random_LTL(3)
    print(print_formula_tree(tree))

((a → c) → b)
~c
~((a & b))
(Gb & (a | a))
~d
(~a & b)
~c
~a
(~a ↔ (a → a))
((b | b) & (c R b))


Translating formula into "pseudo-natural" language

In [220]:
binary_operators = ['&', '|', '→', '↔']
binary_dist = np.array([0.35, 0.35, 0.2, 0.10])

unary_operators = ['G', 'F', 'R', '~']
unary_dist = np.array([0.2, 0.2, 0.2, 0.4])

variables = ['a', 'b', 'c', 'd', 'e', 'f']
variable_dist = np.array([0.3, 0.21, 0.15, 0.13, 0.11, 0.10]) # logarithmic probabilities

symbol2words = {
    '&': "and",
    '|': "or",
    '→': "implies",
    '↔': "iff and only if",
    'G': "it always holds that",
    'F': "finally it holds that", #?????
    'R': "release", #???
    '~': "it's not true that"
}

In [228]:
for i in range(10):
    tree = generate_random_LTL(3)
    formula = print_formula_tree(tree)
    output = ""
    for symbol in formula:
        if symbol in symbol2words.keys():
            output += symbol2words[symbol] + " "

        if symbol in variables:
            output += symbol + " holds "

    print(output)

(a & d) 					 a holds and d holds
(b & b) 					 b holds and b holds
(b | b) 					 b holds or b holds
~b 					 it's not true that b holds
(b & c) 					 b holds and c holds
(d ↔ b) 					 d holds iff and only if b holds
(d & b) 					 d holds and b holds
~d 					 it's not true that d holds
(b ↔ d) 					 b holds iff and only if d holds
(c & a) 					 c holds and a holds


       LTL                         pseudo-langauge
0  (c & a)                     c holds and a holds
1  (d → b)                 if d holds then b holds
2  (a U b)                   a holds until b holds
3  (b | a)                      b holds or a holds
4  (a & b)                     a holds and b holds
5       Fa  eventually it is the case that a holds
6       Ga   globally, it is the case that a holds
7  (c U d)                   c holds until d holds
8  (b | a)                      b holds or a holds
9  (a → a)                 if a holds then a holds
                                               LTL  \
0                                    (b & (c → a))   
1  (((Fa & (b | c)) | (a & b)) | (~((a | b)) | c))   
2             ((b & (Fc | Ga)) → (~((b → b)) | b))   
3                                     (Fa U G(Fc))   
4                        ((a | a) ↔ ((a & b) → d))   
5                          (b U ((a ↔ a) & ~(Gb)))   
6        (~(F((a & b))) | ((Gb | a) | F((b R a))))   
7      